In [1]:
import sys
sys.path.append('../')
import pandas as pd
import random
import os
import numpy as np
import swifter

from cnt.model import (DesignEstimator, RelationExtractor, save_pipeline, load_pipeline, predict_re_single_sentence, 
relations_from_adjectives_df, relations_from_adjectives_single, concat_relations)
from cnt.annotate import (annotate, annotate_single_design, 
                          annotate_designs, 
                          extract_string_from_annotation, labeling_eng)
from cnt.extract_relation import (path, NERTransformer, FeatureExtractor)
from cnt.evaluate import Metrics
from cnt.vectorize import (Doc2Str, Path2Str, Verbs2Str, AveragedPath2Vec, 
                           AveragedRest2Vec)
from cnt.io import (replace_left_right)
from cnt.io import  Database_Connection
from cnt.preprocess import Preprocess


from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline, make_union
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Normalizer
from sklearn.naive_bayes import MultinomialNB
from itertools import product
import warnings
warnings.filterwarnings('ignore')

/home/bigdatalab/anaconda3/envs/nlp/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Database access in text file: "mysql+mysqlconnector://root:YourConnection" -> Format user:password@IP/Database
f = open("/home/bigdatalab/Projects/D4N4/NLP_release_1.0/db_access.txt", "r")
access = f.read()
dc =  Database_Connection(access)

In [3]:
# Define all variables
id_col = "id" 
design_col = "design_en"
language = "_en"
add_columns = ["id", "name"+language, "alternativenames"+language, "class"]
id_col_RE = "design_id"
design_col_RE = "design_en"
ner_model_directory = "../cnt/trained_model/ner/english_new/" # for loading NER
ner_model_name = "english_cno"
re_model_directory = "../cnt/trained_model/re/"  # to save/load RE
re_model_name = "english_cno_new"


In [4]:
# optional - if already defined above
id_col = "id"
design_col = "design_en"
language = "_en"
add_columns = ["id", "name"+language, "alternativenames"+language, "class"]

In [5]:
# Get the annotated designs (ground truth) from the database
# ... under investigation
train = dc.create_own_query("""select design_id, 
(select design_en from nlp_training_designs as nlp where re.design_id=nlp.id) as design_en,
(select name_en from nlp_list_entities as ner where ner.id=re.subject) as s, 
(select class from nlp_list_entities as ner where ner.id=re.subject) as subject_class, 
(select name_en from nlp_list_entities as ner where ner.id=re.predicate) as p, 
(select name_en from nlp_list_entities as ner where ner.id=re.object) as o, 
(select class from nlp_list_entities as ner where ner.id=re.object) as object_class
from nlp_relation_extraction_en_v2 as re;""")

In [6]:
train.head(5)

,design_id,design_en,s,subject_class,p,o,object_class
0,6652,"Aequitas standing left, wearing chiton and him...",Aequitas,PERSON,wearing,chiton,OBJECT
1,6652,"Aequitas standing left, wearing chiton and him...",Aequitas,PERSON,wearing,himation,OBJECT
2,6652,"Aequitas standing left, wearing chiton and him...",Aequitas,PERSON,holding,scales,OBJECT
3,6652,"Aequitas standing left, wearing chiton and him...",Aequitas,PERSON,holding,cornucopia,OBJECT
4,9,Amphora with ribbed surface and crooked handle...,amphora,OBJECT,holding,poppy,PLANT


In [7]:
train.shape

(2390, 7)

In [8]:
# merging s, p, o to one triple into an array of length 1 for each entry - number of rows stay the same
train["y"] = train.apply(lambda row: [(row.s, row.subject_class, row.p, row.o, row.object_class)], axis=1)

In [9]:
train.head(2).style

,design_id,design_en,s,subject_class,p,o,object_class,y
0,6652,"Aequitas standing left, wearing chiton and himation, holding scales and cornucopia.",Aequitas,PERSON,wearing,chiton,OBJECT,"[('Aequitas', 'PERSON', 'wearing', 'chiton', 'OBJECT')]"
1,6652,"Aequitas standing left, wearing chiton and himation, holding scales and cornucopia.",Aequitas,PERSON,wearing,himation,OBJECT,"[('Aequitas', 'PERSON', 'wearing', 'himation', 'OBJECT')]"


In [10]:
# needed for the later merge
tmp = train.groupby("design_id").agg({"y": "sum"})

In [11]:
tmp.loc[tmp.index==1706].style

,y
design_id,
1706,"[('Caracalla', 'PERSON', 'wearing', 'toga', 'OBJECT'), ('Caracalla', 'PERSON', 'holding', 'scroll', 'OBJECT'), ('Geta', 'PERSON', 'wearing', 'toga', 'OBJECT'), ('Geta', 'PERSON', 'holding', 'scroll', 'OBJECT')]"


In [12]:
X = train

In [13]:
# entries for one design are merged --> arrays contain all triples for each design
X = X.merge(tmp, left_on="design_id", right_on="design_id", suffixes=('', 'y'))

In [14]:
X = X[["design_id", "design_en", "yy"]].rename(columns={"yy":"y"})

In [15]:
X.style

,design_id,design_en,y
0,6652,"Aequitas standing left, wearing chiton and himation, holding scales and cornucopia.","[('Aequitas', 'PERSON', 'wearing', 'chiton', 'OBJECT'), ('Aequitas', 'PERSON', 'wearing', 'himation', 'OBJECT'), ('Aequitas', 'PERSON', 'holding', 'scales', 'OBJECT'), ('Aequitas', 'PERSON', 'holding', 'cornucopia', 'OBJECT')]"
1,6652,"Aequitas standing left, wearing chiton and himation, holding scales and cornucopia.","[('Aequitas', 'PERSON', 'wearing', 'chiton', 'OBJECT'), ('Aequitas', 'PERSON', 'wearing', 'himation', 'OBJECT'), ('Aequitas', 'PERSON', 'holding', 'scales', 'OBJECT'), ('Aequitas', 'PERSON', 'holding', 'cornucopia', 'OBJECT')]"
2,6652,"Aequitas standing left, wearing chiton and himation, holding scales and cornucopia.","[('Aequitas', 'PERSON', 'wearing', 'chiton', 'OBJECT'), ('Aequitas', 'PERSON', 'wearing', 'himation', 'OBJECT'), ('Aequitas', 'PERSON', 'holding', 'scales', 'OBJECT'), ('Aequitas', 'PERSON', 'holding', 'cornucopia', 'OBJECT')]"
3,6652,"Aequitas standing left, wearing chiton and himation, holding scales and cornucopia.","[('Aequitas', 'PERSON', 'wearing', 'chiton', 'OBJECT'), ('Aequitas', 'PERSON', 'wearing', 'himation', 'OBJECT'), ('Aequitas', 'PERSON', 'holding', 'scales', 'OBJECT'), ('Aequitas', 'PERSON', 'holding', 'cornucopia', 'OBJECT')]"
4,9,Amphora with ribbed surface and crooked handles containing two ears of corn and poppy.,"[('amphora', 'OBJECT', 'holding', 'poppy', 'PLANT'), ('amphora', 'OBJECT', 'holding', 'corn', 'PLANT')]"
5,9,Amphora with ribbed surface and crooked handles containing two ears of corn and poppy.,"[('amphora', 'OBJECT', 'holding', 'poppy', 'PLANT'), ('amphora', 'OBJECT', 'holding', 'corn', 'PLANT')]"
6,1856,"Anubis advancing right, holding palm branch with right hand over shoulder; in field to left monogram; to right emblem of two conjoined foreparts of prancing horses to right and left. Short ground line.","[('Anubis', 'PERSON', 'holding', 'palm branch', 'OBJECT')]"
7,1582,"Aphrodite standing facing, head right, wearing double chiton and epiblema, holding a long sceptre in right and an apple in outstretched left hand. Border of dots.","[('Aphrodite', 'PERSON', 'wearing', 'chiton', 'OBJECT'), ('Aphrodite', 'PERSON', 'holding', 'scepter', 'OBJECT')]"
8,1582,"Aphrodite standing facing, head right, wearing double chiton and epiblema, holding a long sceptre in right and an apple in outstretched left hand. Border of dots.","[('Aphrodite', 'PERSON', 'wearing', 'chiton', 'OBJECT'), ('Aphrodite', 'PERSON', 'holding', 'scepter', 'OBJECT')]"
9,24725,"Apollo (Bonus Eventus) standing facing, head left, holding patera in outstretched right hand over lighted altar and laurel branch in left hand.","[('Apollo', 'PERSON', 'holding', 'patera', 'OBJECT'), ('Apollo', 'PERSON', 'holding', 'laurel branch', 'OBJECT'), ('Bonus Eventus', 'PERSON', 'holding', 'patera', 'OBJECT'), ('Bonus Eventus', 'PERSON', 'holding', 'laurel branch', 'OBJECT')]"


In [16]:
# duplicate entries for designs are removed (after merge the information is kept in a single row for each design)
X = X.drop_duplicates(['design_id'],keep="first")

In [17]:
# for preprocessing (preprocessing changes the original design text)
X["design_en_changed"] = ""

In [18]:
X.shape

(1250, 4)

In [19]:
df_entities = dc.load_from_db("nlp_list_entities", add_columns)

In [20]:
# Add rules for preprocessing
preprocess = Preprocess()
preprocess.add_rule("horseman", "horse man")
preprocess.add_rule("horsemen", "horse men")

for index, row in df_entities.iterrows():
    if row["alternativenames_en"] is not None:
        if row["class"] != "VERB":
            standard_name = row["name_en"]
            alt_names = row["alternativenames_en"].split(", ")
            for alt_name in alt_names:
                preprocess.add_rule(alt_name, standard_name)

#### Bei römischen Zahlen scheint es noch kleine Probleme zu geben, daher nochmal extra manuell

In [21]:
for rule in list(preprocess.rules):
    if " I." in rule or " II." in rule or " III." in rule or " IV." in rule or " V." in rule:
        del preprocess.rules[rule]

In [22]:
for index, row in X.iterrows():
    if " I." in row["design_en"]:
        X.at[index, "design_en"] = row["design_en"].replace(" I.", " I")
    if " II." in row["design_en"]:
        X.at[index, "design_en"] = row["design_en"].replace(" II.", " II")
    if " III." in row["design_en"]:
        X.at[index, "design_en"] = row["design_en"].replace(" III.", " III")
    if " IV." in row["design_en"]:
        X.at[index, "design_en"] = row["design_en"].replace(" IV.", " IV")
    if " V." in row["design_en"]:
        X.at[index, "design_en"] = row["design_en"].replace(" V.", " V")

## Apply Preprocessing

In [23]:
# Apply defined rules
X["design_en_changed"] = X.swifter.apply(lambda row: preprocess.preprocess_design(row.design_en, row.design_id)[0], axis=1)

Pandas Apply: 100%|█████████████████████████| 1250/1250 [00:23<00:00, 53.28it/s]


In [24]:
# Deleting brackets and questionmarks
X["design_en_changed"] = X.swifter.apply(lambda row: row["design_en_changed"].replace("?", "").replace("(", "").replace(")", ""), axis=1)

Pandas Apply: 100%|█████████████████████| 1250/1250 [00:00<00:00, 195556.88it/s]


In [25]:
X.rename(columns={"design_en":"design_en_orig", "design_en_changed":"design_en", "y":"annotations_orig"}, inplace=True)

In [26]:
# Mapping GT - prepocessing rules are also applied to GT
X["y"] = X.swifter.apply(lambda row: preprocess.preprocess_re(row["annotations_orig"], row.design_id), axis=1)

Pandas Apply: 100%|██████████████████████| 1250/1250 [00:00<00:00, 52609.25it/s]


### Train the RE model

In [27]:
# optional - if already defined above
id_col_RE = "design_id"
design_col_RE = "design_en"

In [28]:
classifier = LogisticRegression(max_iter=1000)
#classifier = RandomForestClassifier()
string_converter = Path2Str(pos=True) 
vectorizer = CountVectorizer(ngram_range=(1,3))
feature = make_pipeline(string_converter, vectorizer)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X[[id_col_RE, design_col_RE]], X[[id_col_RE, "y"]], test_size=0.25, random_state=33)

In [30]:
X.head()

,design_id,design_en_orig,annotations_orig,design_en,y
0,6652,"Aequitas standing left, wearing chiton and him...","[(Aequitas, PERSON, wearing, chiton, OBJECT), ...","Aequitas standing left, wearing chiton and him...","[(Aequitas, PERSON, wearing, chiton, OBJECT), ..."
4,9,Amphora with ribbed surface and crooked handle...,"[(amphora, OBJECT, holding, poppy, PLANT), (am...",Amphora with ribbed surface and crooked handle...,"[(amphora, OBJECT, holding, poppy, PLANT), (am..."
6,1856,"Anubis advancing right, holding palm branch wi...","[(Anubis, PERSON, holding, palm branch, OBJECT)]","Anubis advancing right, holding branch with ri...","[(Anubis, PERSON, holding, palm branch, OBJECT)]"
7,1582,"Aphrodite standing facing, head right, wearing...","[(Aphrodite, PERSON, wearing, chiton, OBJECT),...","Aphrodite standing facing, head right, wearing...","[(Aphrodite, PERSON, wearing, chiton, OBJECT),..."
9,24725,"Apollo (Bonus Eventus) standing facing, head l...","[(Apollo, PERSON, holding, patera, OBJECT), (A...","Apollo Bonus Eventus standing facing, head lef...","[(Apollo, PERSON, holding, patera, OBJECT), (A..."


In [31]:
X.shape

(1250, 5)

#### load pretrained NER-Model

In [32]:
# optional - if already defined above
ner_model_directory = "../cnt/trained_model/ner/english_new/"
ner_model_name = "english_cno"

#### define RE-Model path

In [33]:
# optional - if already defined above
re_model_directory = "../cnt/trained_model/re/"
re_model_name = "english_cno_new"

In [34]:
# training
inner_pipeline = make_pipeline(feature, classifier)
pipeline = make_pipeline(NERTransformer(ner_model_directory, ner_model_name, id_col_RE, design_col_RE),
                         FeatureExtractor(ner_model_directory, ner_model_name, id_col_RE, design_col_RE),
                         RelationExtractor(inner_pipeline, re_model_directory, re_model_name, id_col_RE))
pipeline.fit(X_train, y_train)

Pipeline(steps=[('nertransformer',
                 NERTransformer(design_col='design_en', id_col='design_id',
                                model_dir='../cnt/trained_model/ner/english_new/',
                                model_name='english_cno')),
                ('featureextractor',
                 FeatureExtractor(design_col='design_en', id_col='design_id',
                                  model_dir='../cnt/trained_model/ner/english_new/',
                                  model_name='english_cno')),
                ('relationextractor',
                 RelationExtractor(id_col='design_id',
                                   model_name='english_cno_new',
                                   output_dir='../cnt/trained_model/re/',
                                   pipeline=Pipeline(steps=[('pipeline',
                                                             Pipeline(steps=[('path2str',
                                                                              Path2Str(po

## Save and Load model

In [35]:
save_pipeline(pipeline, re_model_directory, re_model_name)

In [36]:
model = load_pipeline(re_model_directory, re_model_name)

## Predict

In [37]:
y_pred = model.predict(X_test)

In [38]:
metrics = Metrics()

In [39]:
precision, recall = metrics.score_precision_recall(y_test, y_pred)
F1 = (2*precision*recall) / (precision + recall)

In [40]:
print("Precision", round(precision*100,2))
print("Recall", round(recall*100,2))
print("F1", round(F1*100,2))

Precision 84.84
Recall 77.01
F1 80.74


## Map back

In [41]:
# results (that are based on the preprocessed designs) are mapped back to the original designs

X_test["y"] = y_test["y"]

In [42]:
# Deleting brackets and questionmarks
X_test["y_mapped"] = X_test.swifter.apply(lambda row: preprocess.map_re(row["y"], row.design_id), axis=1)

Pandas Apply: 100%|████████████████████████| 313/313 [00:00<00:00, 45809.80it/s]


In [43]:
X_test.head(5).style

,design_id,design_en,y,y_mapped
1915,2491,"Veil Demeter throne left, holding two corn in right hand, left resting on long scepter. Border of dots.","[('Demeter', 'PERSON', 'holding', 'corn', 'PLANT'), ('Demeter', 'PERSON', 'resting_on', 'scepter', 'OBJECT')]","[('Demeter', 'PERSON', 'holding', 'corn', 'PLANT'), ('Demeter', 'PERSON', 'resting_on', 'scepter', 'OBJECT')]"
1196,1661,"Wreath head of Antoninus Pius, right, wearing paludamentum.","[('Antoninus Pius', 'PERSON', 'wearing', 'paludamentum', 'OBJECT')]","[('Antoninus Pius', 'PERSON', 'wearing', 'paludamentum', 'OBJECT')]"
2104,27685,Olybrius resting on shield.,"[('Olybrius', 'PERSON', 'resting_on', 'shield', 'OBJECT')]","[('Olybrius', 'PERSON', 'resting_on', 'shield', 'OBJECT')]"
2312,28041,Ares wearing helmet.,"[('Ares', 'PERSON', 'wearing', 'helmet', 'OBJECT')]","[('Ares', 'PERSON', 'wearing', 'helmet', 'OBJECT')]"
1070,3218,"Labour of Heracles: Nude Heracles standing right, strangling the Nemean lion.","[('Heracles', 'PERSON', 'grasping', 'Nemean lion', 'ANIMAL')]","[('Heracles', 'PERSON', 'grasping', 'Nemean lion', 'ANIMAL')]"


## Auto relations

In [44]:
# adding relations based on adjectives - experimental
obj_list = {
"veiled": ("wearing", "Veil", "before"),
"draped": ("wearing", "Clothing", "before"),
"helmeted": ("wearing", "Helmet", "before"),
"diademed": ("wearing", "Diadem", "before"),
"turreted": ("wearing", "Mural crown", "before"),
"enthroned": ("seated_on", "Throne", "after"),

}

In [45]:
y_pred["design_en"] = X_test.design_en

In [46]:
y_pred = relations_from_adjectives_df(y_pred, "design_en", "y", ner_model_directory, ner_model_name, id_col, design_col, obj_list, entities_to_consider=["PERSON"])

In [47]:
y_pred

,design_id,y,design_en
1915,2491,"[(Demeter, PERSON, holding, corn, PLANT), (Dem...","Veil Demeter throne left, holding two corn in ..."
1196,1661,"[(Antoninus Pius, PERSON, wearing, paludamentu...","Wreath head of Antoninus Pius, right, wearing ..."
2104,27685,"[(Olybrius, PERSON, resting_on, shield, OBJECT)]",Olybrius resting on shield.
2312,28041,"[(Ares, PERSON, wearing, helmet, OBJECT)]",Ares wearing helmet.
1070,3218,"[(Heracles, PERSON, grasping, Nemean lion, ANI...",Labour of Heracles: Nude Heracles standing rig...
...,...,...,...
1628,944,[],"Perseus and Andromeda; at left, Andromeda stan..."
922,341,[],"Head of Demeter facing, slightly right, hair i..."
2253,27752,"[(Olybrius, PERSON, seated_on, kline, OBJECT)]",Olybrius seated on kline.
2287,27541,"[(Olybrius, PERSON, seated_on, shield, OBJECT)]",Olybrius seated on shield.


In [48]:
# for testing a single design

design = "Diademed Athena to the left and helmeted Ares to the right, holding sword."
# rule based - see above 
auto_relations = relations_from_adjectives_single(design,ner_model_directory, ner_model_name, id_col_RE, design_col_RE, obj_list)
# results of the model
model_relations = predict_re_single_sentence(model, design, id_col_RE, design_col_RE)
# merging both results
concat_relations(auto_relations, model_relations)

[('Ares', 'PERSON', 'wearing', 'Helmet', 'OBJECT'),
 ('Athena', 'PERSON', 'wearing', 'Diadem', 'OBJECT'),
 ('Athena', 'PERSON', 'holding', 'sword', 'OBJECT'),
 ('Ares', 'PERSON', 'holding', 'sword', 'OBJECT')]

In [55]:
# optional -only if results need to be uploaded to database
# preprocessing
upload = True
f = open("/home/bigdatalab/Projects/D4N4/NLP_release_1.0/db_access.txt", "r")
access = f.read()
dc =  Database_Connection(access)
cnt_designs = dc.load_designs_from_db("data_designs", [id_col, design_col])
cnt_designs = cnt_designs.rename(columns={"id": "design_id"})

cnt_designs["design_en_changed"] = cnt_designs.swifter.apply(lambda row: preprocess.preprocess_design(row.design_en, row.design_id)[0], axis=1)
cnt_designs["design_en_changed"] = cnt_designs.swifter.apply(lambda row: row["design_en_changed"].replace("?", "").replace("(", "").replace(")", ""), axis=1)

Pandas Apply: 100%|█████████████████████| 7618/7618 [00:00<00:00, 211824.27it/s]


In [56]:
# optional -only if results need to be uploaded to database
cnt_designs = cnt_designs.rename(columns={"design_en": "design_en_org"})
cnt_designs = cnt_designs.rename(columns={"design_en_changed": "design_en"})

In [57]:
# optional -only if results need to be uploaded to database - TODO: check by Sebastian on BDL-Server
upload = True
if upload ==True:
    cnt_pred = model.predict(cnt_designs)
    cnt_pipeline_output = pd.DataFrame([(str(designid), *relation) for  _, (designid, relation_list) in cnt_pred.iterrows()
                    for relation in relation_list],
            columns=["DesignID", "Person", "Label_Person", "Relation", "Object",
                     "Label_Object"])
    cnt_pipeline_output.to_sql("cnt_pipeline_output",dc.mysql_connection,if_exists="replace", index=False)

In [58]:
cnt_designs.head()

,design_id,design_en_org,design_en
0,1,Diademed head of deified Alexander the Great w...,Diadem head of deified Alexander the Great wit...
1,6,"Altar, lighted and garlanded.","Altar, lighted and garland."
2,8,Prize amphora on ornamental stand; within line...,Prize amphora on ornamental stand; within line...
3,9,Amphora with ribbed surface and crooked handle...,Amphora with ribbed surface and crooked handle...
4,10,"Bust of youthful Anchialos, right, wearing tae...","Bust of youthful Anchialos, right, wearing tae..."
